In [1]:
from os.path import realpath
import glob
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
import join as join
import seaborn as sns
import tensorflow as tf
import shutil

from sklearn.ensemble import RandomForestClassifier

from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2022-10-25 13:04:10.731097: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-25 13:04:10.868852: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-25 13:04:11.374687: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.10/dist-packages/cv2/../../lib64:
2022-10-25 13:04:11.374797: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerro

# Iremos descobrir qual foto tem qual classe através da concatenação dos dataset

In [2]:
dados = pd.read_csv('./dados.csv')
path_interno = pd.read_csv('./path_interno.csv')
path_externo = pd.read_csv('./path_externo.csv')

In [3]:
path1 = '/home/luan/Desktop/feijao/Classificacao-main/images/interna'
path2 = '/home/luan/Desktop/feijao/Classificacao-main/images/externa'

In [4]:
dados.rename(columns={"REPETIÇÃO":"REPETICAO"}, inplace=True)

In [5]:
path_interno['SEMENTE'] = path_interno['SEMENTE'].str.replace(".jpg", "")
path_externo['SEMENTE'] = path_externo['SEMENTE'].str.replace(".jpg", "")

/tmp/ipykernel_101198/3216806358.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  path_interno['SEMENTE'] = path_interno['SEMENTE'].str.replace(".jpg", "")
/tmp/ipykernel_101198/3216806358.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  path_externo['SEMENTE'] = path_externo['SEMENTE'].str.replace(".jpg", "")


In [6]:
path_interno['SEMENTE'] = path_interno['SEMENTE'].astype(int)
path_externo['SEMENTE'] = path_externo['SEMENTE'].astype(int)
path_interno['LOTE'] = path_interno['LOTE'].astype(int)
path_externo['LOTE'] = path_externo['LOTE'].astype(int)
path_interno['REPETICAO'] = path_interno['REPETICAO'].astype(int)
path_externo['REPETICAO'] = path_externo['REPETICAO'].astype(int)

In [7]:
path_interno = dados.merge(path_interno, how  = "right")
path_externo = dados.merge(path_externo, how  = "right")

In [8]:
path_interno[path_interno['CLASSE'] == 1]

,LOTE,REPETICAO,SEMENTE,CLASSE,VIGOR,PATH
284,4,2,42,1,0,/home/luan/Desktop/feijao/Classificacao-main/i...
418,1,2,41,1,0,/home/luan/Desktop/feijao/Classificacao-main/i...
423,1,2,47,1,0,/home/luan/Desktop/feijao/Classificacao-main/i...
443,1,2,50,1,0,/home/luan/Desktop/feijao/Classificacao-main/i...


In [9]:
list = path_interno['CLASSE'].unique()

In [10]:
labels = path_interno.sort_values('CLASSE')

In [11]:
for i in list:
    os.makedirs(os.path.join(path1, f"classe_{str(i)}"))

In [12]:
for i in path_interno.itertuples(index=False):
    print(i[3],i[5])

4 /home/luan/Desktop/feijao/Classificacao-main/imagens_cortadas/LOTE_INTERNO_2_R2/LOTE_INTERNO_2_R2_semente_32.jpg
2 /home/luan/Desktop/feijao/Classificacao-main/imagens_cortadas/LOTE_INTERNO_2_R2/LOTE_INTERNO_2_R2_semente_13.jpg
3 /home/luan/Desktop/feijao/Classificacao-main/imagens_cortadas/LOTE_INTERNO_2_R2/LOTE_INTERNO_2_R2_semente_37.jpg
4 /home/luan/Desktop/feijao/Classificacao-main/imagens_cortadas/LOTE_INTERNO_2_R2/LOTE_INTERNO_2_R2_semente_35.jpg
4 /home/luan/Desktop/feijao/Classificacao-main/imagens_cortadas/LOTE_INTERNO_2_R2/LOTE_INTERNO_2_R2_semente_47.jpg
4 /home/luan/Desktop/feijao/Classificacao-main/imagens_cortadas/LOTE_INTERNO_2_R2/LOTE_INTERNO_2_R2_semente_6.jpg
3 /home/luan/Desktop/feijao/Classificacao-main/imagens_cortadas/LOTE_INTERNO_2_R2/LOTE_INTERNO_2_R2_semente_5.jpg
4 /home/luan/Desktop/feijao/Classificacao-main/imagens_cortadas/LOTE_INTERNO_2_R2/LOTE_INTERNO_2_R2_semente_2.jpg
3 /home/luan/Desktop/feijao/Classificacao-main/imagens_cortadas/LOTE_INTERNO_2_R2/L

In [13]:
for c in list:
    for i in path_interno.itertuples(index=False):
        #create path to the image 
        if(c == i[3]):
            name = str(i[5]).split("/")[-1]
            get_image = os.path.join(str(i[5]))
            #If image has not already exist in the new folder create one        
            if not os.path.exists(f'{path1}/classe_{c}/'+name):
                # move the image 
                move_image_to_cat = shutil.copy(get_image, f'{path1}/classe_{c}/')


In [14]:
for i in list:
    os.makedirs(os.path.join(path2, f"classe_{str(i)}"))

In [15]:
path_externo.sort_values("CLASSE", inplace=True)

In [16]:
for c in list:
    for i in path_externo.itertuples(index=False):
        #create path to the image 
        if(c == i[3]):
            name = str(i[5]).split("/")[-1]
            get_image = os.path.join(str(i[5]))
            #If image has not already exist in the new folder create one        
            if not os.path.exists(f'{path2}/classe_{c}/'+name):
                # move the image 
                move_image_to_cat = shutil.copy(get_image, f'{path2}/classe_{c}/')